**This notebook extracts and merges the Geospacial data with the Neighbourhood data in a DataFrame**

> Import all the modules

In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BSoup
"""
Not working:
import geocoder # import geocoder
from geopy.geocoders import Nominatim
"""

'\nNot working:\nimport geocoder # import geocoder\nfrom geopy.geocoders import Nominatim\n'

> Extract the Neighbourhood data from the Wikipedia page and save it in a DataFrame

In [0]:
BASE_URI = 'https://en.wikipedia.org'
page = requests.get(BASE_URI+'/wiki/List_of_postal_codes_of_Canada:_M')

# Using BeautifulSoup to make fetch happen
soup = BSoup(page.content, 'html.parser')

# Picking the target elements
neighbourhood_on_page = soup.select('table.wikitable tbody tr')

# Extracting values from target elements:
neighbourhood_data = list()
for neighbourhood in neighbourhood_on_page:
  neighbour_dict = dict()
  element_chain = list()
  for child in neighbourhood.children:
    if 'get_text' in dir(child):
      element = child.get_text()
    else:
      element = child.string
    element_chain.append(element)

  # Cleaning: encoding strings and replacing newline characters from returned HTML leaves
  element_chain = [element for element in map(lambda chain: str(chain).replace('\n', ''), element_chain)]

  # Cleaning: removing escape characters from returned list of elements
  elements = [element for element in filter(lambda element: element, element_chain)]
  if elements[1] != 'Not assigned': 
    # Filtering "Not assigned" Boroughs
    if elements[2] == 'Not assigned':
      # Cleaning "Not assigned" Neighbourhoods
      elements[2] = elements[1]
    neighbourhood_data.append(elements)

df_cols = neighbourhood_data.pop(0)

neighbourhood_df = pd.DataFrame(neighbourhood_data, columns=df_cols)

grouped_df = pd.DataFrame({'Neighbourhood':neighbourhood_df.groupby('Postcode').apply(lambda x: ','.join(x.Neighbourhood))})
grouped_df.reset_index(inplace=True)

> Since the geospatial data could not be extracted from the libraries: the **csv** file has been used to get the latitudes and longitudes of the places.

> Finally, the geospatial values are merged with our neighbourhood data in a DataFrame.

In [0]:
# Fetching Geospatial Data from URL:
data_csv = requests.get('https://cocl.us/Geospatial_data').text.split('\r\n')
data = [obj for obj in map(lambda i: i.split(','), data_csv)]
cols = data.pop(0)
cols[0] = df_cols[0]
geodata_df = pd.DataFrame(data, columns=cols)

# Unifying the repeated Neighbourhood values of the same Postalcode
merged_df = pd.merge(neighbourhood_df[['Postcode', 'Borough']], grouped_df, how="inner", on="Postcode")
merged_df.drop_duplicates(inplace=True)

# Merging the neighbourhood data with their geospatial values
final_df = pd.merge(merged_df, geodata_df, how="inner", on="Postcode")
print(final_df)